# Starbucks Worldwide Location Project

# CRISP-DM

# A. Business Understanding

**Business Objective**: The main objective of the project is to develop a predictive model that can accurately estimate the number of Starbucks stores in a country based on a set of variables, such as GDP per capita, population, and easy of doing business. 

**Business Success Criteria**: For this project, the business success criteria have at least a 0.7 R-Squared value for the model's predictions.

---

**Dataset**: Two datasets available on Kaggle were used for Starbucks numbers in countries. The first of them belongs to 2016, the other belongs to 2021. In addition, World Bank data were used for parameters such as GPD and population of countries. World Bank data was obtained with the wbgapi library.

[Starbucks Locations Worldwide 2016](https://www.kaggle.com/datasets/starbucks/store-locations)

[Starbucks Locations Worldwide 2021](https://www.kaggle.com/datasets/kukuroo3/starbucks-locations-worldwide-2021-version)

__Important note:__ _The Starbucks data set for 2021 was used only in the data analysis part with certain assumptions. The machine learning model was made according to the data set in 2016 and the World Bank data._

---

**Technologies and Tools**:

* Programming Language:
    * Python
* Tools:
    * Jupyter Notebook
* Libraries
    * Pandas
    * Numpy
    * Scikit-learn
    * LighGBM
    * CatBoost
    * XGBoost

Since the problem we are trying to solve involves predicting a continuous outcome, I will use regression models in both classical and relatively new machine learning models. This will allow us to explore different modeling approaches and compare their performance.

List of models I used in this project as a result of my research:
* Machine Learning Models
    * Linear Regression
    * Decision Tree Regression
    * Random Forest Regression
    * Boost Methods
        * XGBoost
        * LightGBM
        * CatBoost


# B. Data Understanding 

## Install Libraries

In [ ]:
import numpy as np
import pandas as pd

import pycountry as pc
import pycountry_convert as pcc
import wbgapi as wb

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, RepeatedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

## Collect Initial Data

### Starbucks Worldwide Locations

In [ ]:
starbucks_location_2016 = pd.read_csv('data/starbucks_locations_2016.csv', sep= ',')
starbucks_location_2021 = pd.read_csv('data/starbucks_locations_2022.csv', sep= ',')

### World Bank Dataset

## Describe Data

### Check Duplicate Value

### Check Empty Value

# C. Data Preparation

# D. Modelling

# E. Evaluation